# MIS-CPIMS Data Import Scripts

Import pandas library and read exported CPARA data from MIS (XLSX). Display all columns for exploration.

In [1]:
import pandas as pd
cpara=pd.read_excel("Data/CPARA.xlsx", sheet_name="Sheet1")
cpara_ovc_questions = pd.read_excel("Data/OVC_Questions.xlsx", sheet_name="Sheet1")
cpims_hhs=pd.read_excel("Data/Household ID OVC Mapping.xlsx", sheet_name="Sheet1")
cpara.columns

Index(['UnIque ID', 'record_id', 'assessment_date', 'cpims_id', 'case_manager',
       'child_headed', 'has_hei', 'has_pbf', 'has_svac', 'q1_1', 'q1_2',
       'q1_3', 'q1_4', 'benchmark1', 'q2_1', 'q2_2', 'q2_3', 'q2_4', 'q2_5',
       'q2_6', 'q2_8', 'q2_7', 'q2_9', 'benchmark2', 'q3_1', 'q3_2', 'q3_3',
       'benchmark3', 'q4_1', 'q4_2', 'q4_3', 'q4_4', 'benchmark4', 'q5_1',
       'q5_2', 'q5_3', 'benchmark5', 'q6_1', 'q6_2', 'q6_3', 'q6_4', 'q6_5',
       'benchmark6', 'q7_1', 'q7_2', 'benchmark7', 'q8_1', 'benchmark8',
       'q9_1', 'q9_2', 'q9_3', 'q9_4', 'benchmark9', 'healthy', 'stable',
       'safe', 'schooled', 'entry_time', 'action_by', 'designation', 'lip',
       'entry_by_name', 'client_time', 'deleted', 'modified', 'gps', 'device',
       'cpimsId', 'names', 'caregiverhivstatus', 'cbo_id', 'cbo', 'county',
       'subcounty', 'ward', 'chv_id', 'chv'],
      dtype='object')

In [30]:
import uuid

print(uuid.uuid4())

record_counts = cpara['record_id'].count()



def generate_uid(count):
    uid_list = []
    for i in range (0, count):
        uuid_gen = uuid.uuid4()
        print(f'generated uid: {uuid_gen} inte: {i}')
        uid_list.append(uuid_gen)
    return uid_list
    

df_uid = pd.DataFrame(generate_uid(record_counts))

5f5f1781-2e88-4f11-93b1-2bde1f6d91d7
generated uid: 701114ae-e034-4bf9-9f4a-0ba2a48a8dce inte: 0
generated uid: 7c1764b1-9ecb-4dd0-8d84-1f1c664a9a89 inte: 1
generated uid: 98a582fb-7be6-4494-a08b-95eb2e4d4c67 inte: 2
generated uid: 97b59efe-7495-4d68-b3e6-02818661c393 inte: 3
generated uid: c74c84ec-5f04-4a40-aaa7-7140316f3222 inte: 4
generated uid: 7497f3e8-2a41-4fe1-a1e3-d7247f8a5302 inte: 5
generated uid: 50b586ec-fca7-4b39-a98f-98b02f7f1a65 inte: 6
generated uid: d0450bff-15c7-45ce-aee5-f482a8162ecf inte: 7
generated uid: fbb5b07d-8b8e-4eef-8ae7-bb88f0e43110 inte: 8
generated uid: b53a6fd5-514e-410e-b777-6b5ff7be4189 inte: 9
generated uid: 3f0f33c8-4c26-4e6b-9ab5-b177716f1ecc inte: 10
generated uid: 40c820fc-2e2a-4e50-a2ba-1f40bb2df807 inte: 11
generated uid: 9d35a073-af9a-4352-8a48-b16de76d38f8 inte: 12
generated uid: 536bb4d0-bd64-4be2-933d-b19ad7865a68 inte: 13
generated uid: 6da828ac-915d-44bb-9ce0-e9ce9bdcdcde inte: 14
generated uid: adb7876a-03af-4ade-b1ed-b2b2a7240481 inte: 

In [31]:
print(df_uid)

                                          0
0      701114ae-e034-4bf9-9f4a-0ba2a48a8dce
1      7c1764b1-9ecb-4dd0-8d84-1f1c664a9a89
2      98a582fb-7be6-4494-a08b-95eb2e4d4c67
3      97b59efe-7495-4d68-b3e6-02818661c393
4      c74c84ec-5f04-4a40-aaa7-7140316f3222
...                                     ...
32555  a4453e61-b2c3-4a6d-82e9-54e0c629ccdf
32556  2a26aa94-7909-46f8-9b5d-35c288dc87bb
32557  d29044fb-5b78-4073-adf7-f8a4efd24e8e
32558  1230f5cc-170f-4e5b-afde-61b8f11c74af
32559  756da716-b356-48e4-9d40-ea7e9a2df30d

[32560 rows x 1 columns]


In [13]:
cpims_questions=pd.read_excel("Data/ovc_care_questions.xlsx", sheet_name="Sheet1")

In [33]:
cpara_data=pd.merge(cpara, cpims_hhs, how='left', left_on='cpims_id', right_on='caregiver_id')
cpara_data1 = pd.merge(cpara_data, df_uid, left_index=True, right_index=True)
cpara_data1.head(2)

MergeError: No common columns to perform merge on. Merge options: left_on=None, right_on=None, left_index=False, right_index=False

In [3]:
cpara_ovc_questions.columns

Index(['record_id', 'cpara_record_id', 'cpara_date', 'question', 'response',
       'cpims_id', 'caregiver_id', 'chv_id'],
      dtype='object')

In [4]:
cpara_individual=pd.merge(cpara_ovc_questions, cpims_hhs, how='left', left_on='caregiver_id', right_on='caregiver_id')
cpara_individual.head(2)

,record_id,cpara_record_id,cpara_date,question,response,cpims_id,caregiver_id,chv_id,cpims_ovc_id,household
0,5,8859,2022-08-02,q3.1,Yes,1705501,1676623,2042693,1687650.0,b540b485-7399-4407-bd14-066f5b79c133
1,5,8859,2022-08-02,q3.1,Yes,1705501,1676623,2042693,1705501.0,b540b485-7399-4407-bd14-066f5b79c133


In [5]:
cpara_ovc_questions.dtypes #cpims_hhs.dtypes

record_id           int64
cpara_record_id     int64
cpara_date         object
question           object
response           object
cpims_id            int64
caregiver_id        int64
chv_id              int64
dtype: object

Align the MIS data from rows to columns => vertical and only display the first two records. Move the columns that should not be considered as var.

In [6]:
cpara_unpivot=pd.melt(cpara_data, id_vars=['record_id', 'assessment_date', 'cpims_id', 'entry_time', 'action_by',
       'designation', 'lip', 'entry_by_name','client_time', 'modified', 'deleted', 'gps', 'device', 'case_manager', 'cpims_ovc_id', 'household'])

cpara_unpivot.head(2)

,record_id,assessment_date,cpims_id,entry_time,action_by,designation,lip,entry_by_name,client_time,modified,deleted,gps,device,case_manager,cpims_ovc_id,household,variable,value
0,9325,2022-09-12,105151,2022-09-27 12:49:56,3180741,Data Assistant,MAKE ME SMILE,ZILPA AKINYI,2022-09-27 13:49:49,2022-09-27 12:49:56,NaN,NaN,screen=1440 x 900;browser=Chrome;browserVersio...,NaN,4282788.0,ae1ff881-bf28-471a-9a03-3efceed4d96a,UnIque ID,2022-09-12-105151
1,9325,2022-09-12,105151,2022-09-27 12:49:56,3180741,Data Assistant,MAKE ME SMILE,ZILPA AKINYI,2022-09-27 13:49:49,2022-09-27 12:49:56,NaN,NaN,screen=1440 x 900;browser=Chrome;browserVersio...,NaN,4282807.0,ae1ff881-bf28-471a-9a03-3efceed4d96a,UnIque ID,2022-09-12-105151


In [7]:
cpara_unpivot.columns

Index(['record_id', 'assessment_date', 'cpims_id', 'entry_time', 'action_by',
       'designation', 'lip', 'entry_by_name', 'client_time', 'modified',
       'deleted', 'gps', 'device', 'case_manager', 'cpims_ovc_id', 'household',
       'variable', 'value'],
      dtype='object')

In [8]:
ovc_care_benchmark = cpara_unpivot[cpara_unpivot["variable"].str.contains('benchmark')].pivot(index=['record_id', 'cpims_id','assessment_date','entry_time', 'modified', 'cpims_ovc_id', 'household'], columns='variable', values=['value'])
ovc_care_benchmark.to_excel("ovc_care_benchmark.xlsx")
ovc_care_benchmark.head(2)

value  \
variable                                                                                          benchmark1   
record_id cpims_id assessment_date entry_time          modified            cpims_ovc_id household              
1         3273808  2022-09-07      2022-09-08 08:33:03 2022-09-08 08:33:03 NaN          NaN              Yes   
51        3075340  2022-09-06      2022-09-08 14:19:54 2022-09-08 14:19:54 NaN          NaN              Yes   

                                                                                                              \
variable                                                                                          benchmark2   
record_id cpims_id assessment_date entry_time          modified            cpims_ovc_id household              
1         3273808  2022-09-07      2022-09-08 08:33:03 2022-09-08 08:33:03 NaN          NaN              Yes   
51        3075340  2022-09-06      2022-09-08 14:19:54 2022-09-08 14:19:54 NaN          NaN              Yes   

                                                                                                              \
variable                                                                                          benchmark3   
record_id cpims_id assessment_date entry_time          modified            cpims_ovc_id household              
1         3273808  2022-09-07      2022-09-08 08:33:03 2022-09-08 08:33:03 NaN          NaN              Yes   
51        3075340  2022-09-06      2022-09-08 14:19:54 2022-09-08 14:19:54 NaN          NaN              Yes   

                                                                                                              \
variable                                                                                          benchmark4   
record_id cpims_id assessment_date entry_time          modified            cpims_ovc_id household              
1         3273808  2022-09-07      2022-09-08 08:33:03 2022-09-08 08:33:03 NaN          NaN              Yes   
51        3075340  2022-09-06      2022-09-08 14:19:54 2022-09-08 14:19:54 NaN          NaN              Yes   

                                                                                                              \
variable                                                                                          benchmark5   
record_id cpims_id assessment_date entry_time          modified            cpims_ovc_id household              
1         3273808  2022-09-07      2022-09-08 08:33:03 2022-09-08 08:33:03 NaN          NaN               No   
51        3075340  2022-09-06      2022-09-08 14:19:54 2022-09-08 14:19:54 NaN          NaN               No   

                                                                                                              \
variable                                                                                          benchmark6   
record_id cpims_id assessment_date entry_time          modified            cpims_ovc_id household              
1         3273808  2022-09-07      2022-09-08 08:33:03 2022-09-08 08:33:03 NaN          NaN              Yes   
51        3075340  2022-09-06      2022-09-08 14:19:54 2022-09-08 14:19:54 NaN          NaN              Yes   

                                                                                                              \
variable                                                                                          benchmark7   
record_id cpims_id assessment_date entry_time          modified            cpims_ovc_id household              
1         3273808  2022-09-07      2022-09-08 08:33:03 2022-09-08 08:33:03 NaN          NaN              Yes   
51        3075340  2022-09-06      2022-09-08 14:19:54 2022-09-08 14:19:54 NaN          NaN              Yes   

                                                                                                              \
variable                                                        

In [9]:
#ovc_care_cpara = cpara_unpivot[cpara_unpivot["variable"].str.startswith('q')]
ovc_care_cpara = cpara_unpivot[cpara_unpivot["variable"].str.startswith('q')].pivot(index=['record_id', 'cpims_id','assessment_date','cpims_ovc_id', 'household'], columns='variable', values=['value'])
ovc_care_cpara.to_csv("ovc_care_cpara_pivot.csv")
ovc_care_cpara.head(2)

value            \
variable                                                   q1_1 q1_2 q1_3   
record_id cpims_id assessment_date cpims_ovc_id household                   
1         3273808  2022-09-07      NaN          NaN          na  Yes   na   
51        3075340  2022-09-06      NaN          NaN          na  Yes   na   

                                                                               \
variable                                                  q1_4 q2_1 q2_2 q2_3   
record_id cpims_id assessment_date cpims_ovc_id household                       
1         3273808  2022-09-07      NaN          NaN        Yes   na   na   na   
51        3075340  2022-09-06      NaN          NaN        Yes   na   na   na   

                                                                          ...  \
variable                                                  q2_4 q2_5 q2_6  ...   
record_id cpims_id assessment_date cpims_ovc_id household                 ...   
1         3273808  2022-09-07      NaN          NaN         na   na   na  ...   
51        3075340  2022-09-06      NaN          NaN         na   na   na  ...   

                                                                               \
variable                                                  q6_3 q6_4 q6_5 q7_1   
record_id cpims_id assessment_date cpims_ovc_id household                       
1         3273808  2022-09-07      NaN          NaN         No   na   na  Yes   
51        3075340  2022-09-06      NaN          NaN         No   na   na  Yes   

                                                                               \
variable                                                  q7_2 q8_1 q9_1 q9_2   
record_id cpims_id assessment_date cpims_ovc_id household                       
1         3273808  2022-09-07      NaN          NaN        Yes  Yes  Yes  Yes   
51        3075340  2022-09-06      NaN          NaN        Yes  Yes  Yes   No   

                                                                     
variable                                                  q9_3 q9_4  
record_id cpims_id assessment_date cpims_ovc_id household            
1         3273808  2022-09-07      NaN          NaN         na  Yes  
51        3075340  2022-09-06      NaN          NaN         na  Yes  

[2 rows x 35 columns]

In [10]:
cpara_individual.to_excel("cpara_ovc_questions.xlsx")
cpara_individual.head(2)

,record_id,cpara_record_id,cpara_date,question,response,cpims_id,caregiver_id,chv_id,cpims_ovc_id,household
0,5,8859,2022-08-02,q3.1,Yes,1705501,1676623,2042693,1687650.0,b540b485-7399-4407-bd14-066f5b79c133
1,5,8859,2022-08-02,q3.1,Yes,1705501,1676623,2042693,1705501.0,b540b485-7399-4407-bd14-066f5b79c133


In [11]:
cpims_hhs.columns

Index(['cpims_ovc_id', 'caregiver_id', 'household'], dtype='object')

In [108]:
cpara_data.to_excel("CPARA_MAPPING.xlsx")

In [109]:
cpara_data.head(2)

,UnIque ID,record_id,assessment_date,cpims_id,case_manager,child_headed,has_hei,has_pbf,has_svac,q1_1,...,cbo_id,cbo,county,subcounty,ward,chv_id,chv,cpims_ovc_id,caregiver_id,household
0,2022-09-12-105151,9325,2022-09-12,105151,NaN,No,No,No,No,Yes,...,2796.0,MAKE ME SMILE,Kisumu,Kisumu East,Manyatta 'B',3173240.0,GILLIANN GILLIAN ACHIENGGG,4282788.0,105151.0,ae1ff881-bf28-471a-9a03-3efceed4d96a
1,2022-09-12-105151,9325,2022-09-12,105151,NaN,No,No,No,No,Yes,...,2796.0,MAKE ME SMILE,Kisumu,Kisumu East,Manyatta 'B',3173240.0,GILLIANN GILLIAN ACHIENGGG,4282807.0,105151.0,ae1ff881-bf28-471a-9a03-3efceed4d96a
